## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!proxychains wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

[proxychains] config file found: /etc/proxychains.conf
[proxychains] preloading /usr/lib/libproxychains4.so
[proxychains] DLL init: proxychains-ng 4.13
--2018-12-11 14:19:54--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www.dropbox.com (www.dropbox.com)... 224.0.0.1
Connecting to www.dropbox.com (www.dropbox.com)|224.0.0.1|:443... [proxychains] Strict chain  ...  127.0.0.1:1080  ...  www.dropbox.com:443  ...  OK
connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2018-12-11 14:19:56--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3e31b120a3f4fc54b8d3bc32db.dl.dropboxusercontent.com/cd/0/get/AXP4a3A0X_q_Z_ZkTLLXQCGZuQQC6ZdCEF6jcSHDtgrjHHSmjjxwy7XmQ4KGomRwINWSrqSaawaJ6ktaySa_jxxDojb6EQhT1CdptDUBIafQlA

In [3]:
import numpy as np

data = list(open("./quora.txt"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [5]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [6]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(sentence.lower()) for sentence in data]

In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [8]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [9]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [10]:
# now you can get word vectors !
model.get_vector('anything')

array([-4.1129956 , -1.1661212 ,  2.8884234 ,  1.6030418 , -2.3407958 ,
       -2.508324  ,  2.2683372 ,  0.55404526, -1.0120211 ,  0.34297723,
        0.30048978, -0.72767496, -0.1630052 ,  0.9826463 , -0.33899012,
       -0.09860088, -0.0649424 , -1.3995548 ,  1.0637907 , -2.746431  ,
        4.2340827 ,  0.26218152, -0.8690755 , -0.32636678,  2.3271887 ,
        2.0019982 ,  1.9384661 , -2.387909  ,  4.108293  ,  2.54925   ,
        2.4769561 ,  1.8132794 ], dtype=float32)

In [11]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

/home/yulin_shen/anaconda3/envs/nlp/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cheese', 0.9462973475456238),
 ('rice', 0.939405083656311),
 ('butter', 0.9346649646759033),
 ('sauce', 0.9208490252494812),
 ('fruit', 0.9111307859420776),
 ('wine', 0.9024859666824341),
 ('chocolate', 0.9022200107574463),
 ('grass', 0.9016539454460144),
 ('potato', 0.9014739394187927),
 ('banana', 0.8996329307556152)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [12]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [20]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843312203884125),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [75]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [76]:
# for each word, compute it's vector with model
word_vectors = np.asarray([model.get_vector(word) for word in words])

In [77]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [93]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

# and maybe MORE OF YOUR CODE here :)
word_vectors_pca -= word_vectors_pca.mean(0)
word_vectors_pca /= word_vectors_pca.std(0)

In [94]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [95]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [96]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [100]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE(verbose=100).fit_transform(word_vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.009s...
[t-SNE] Computed neighbors for 1000 samples in 0.274s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.085s
[t-SNE] Iteration 50: error = 70.0051498, gradient norm = 0.2970318 (50 iterations in 13.194s)
[t-SNE] Iteration 100: error = 68.6086731, gradient norm = 0.3093194 (50 iterations in 12.680s)
[t-SNE] Iteration 150: error = 69.0293121, gradient norm = 0.2983782 (50 iterations in 13.917s)
[t-SNE] Iteration 200: error = 70.0661850, gradient norm = 0.2864605 (50 iterations in 13.147s)
[t-SNE] Iteration 250: error = 68.9467392, gradient norm = 0.2919677 (50 iterations in 13.747s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 68.946739
[t-SNE] Iteration 300: error = 1.1905084, gradient norm = 0.0019985 (50 iterations in 10.519s)
[t-SNE] Iteration 350: error = 1.0992552, gradient norm = 0

In [101]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1230', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [110]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # YOUR CODE
    phrase = phrase.lower()
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(phrase)
    count = 0
    for token in tokens:
        if token in model:
            vector += model.get_vector(token)
            count += 1
    return vector / count

In [111]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [113]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.asarray([get_phrase_embedding(chosen_phrase) for chosen_phrase in chosen_phrases])

In [114]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [115]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.011s...
[t-SNE] Computed neighbors for 1001 samples in 0.415s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.477874
[t-SNE] Computed conditional probabilities in 0.113s
[t-SNE] Iteration 50: error = 77.2433624, gradient norm = 0.3723142 (50 iterations in 14.027s)
[t-SNE] Iteration 100: error = 79.3762512, gradient norm = 0.3158246 (50 iterations in 15.334s)
[t-SNE] Iteration 150: error = 81.2811050, gradient norm = 0.2739211 (50 iterations in 16.732s)
[t-SNE] Iteration 200: error = 82.1632690, gradient norm = 0.2965964 (50 iterations in 16.844s)
[t-SNE] Iteration 250: error = 80.7172852, gradient norm = 0.2912118 (50 iterations in 17.562s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 80.717285
[t-SNE] Iteration 300: error = 2.1303499, gradient norm = 0.0027274 (50 iterations in 12

In [116]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1357', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [117]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

/home/yulin_shen/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [186]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_vector = get_phrase_embedding(query)
    similar_list = np.dot(data_vectors, query_vector) / np.linalg.norm(data_vectors, axis=1) / np.linalg.norm(query_vector)
    topk_index = np.argpartition(-similar_list, k)[:k]
    topk_order = np.argsort(-similar_list[topk_index])
    selected_data = [data[index] for index in topk_index]
    return [selected_data[order] for order in topk_order]

In [187]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [188]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [190]:
find_nearest(query="Why don't i ask a question myself?", k=10)

['Why Aricent?\n',
 'lol ?\n',
 'Do we really need AI? Why yes? Why no?\n',
 'What does "fuck" mean?\n',
 'Why IOCTL fail?\n',
 'HOW DOESI FUCK A LADY?\n',
 'Why do people fuck?\n',
 'Why does xsandy.de play so nasty?\n',
 'What was Suck.com?\n',
 'What doesاستشفر mean?\n']

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.